# Denoising Diffusion Probabilistic Models

In [50]:
import torch 
import torch.nn as nn

In [51]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        print(x.shape)
        x = self.relu(self.conv1(x))
        print(x.shape)
        print("-"*10)
        return self.relu(self.conv2(x))
    
class DownBlock(nn.Module):
    def __init__(self, filters, in_channels):
        super(DownBlock, self).__init__()
        conv_blocks = [ConvBlock(in_channels, filters[0])]
        for i in range(1, len(filters)):
            conv_blocks.append(ConvBlock(filters[i-1], filters[i]))

        self.conv_blocks = nn.Sequential(*conv_blocks)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        residual_outputs = []
        for conv_block in self.conv_blocks:
            x = conv_block(x)
            residual_outputs.append(x)
            x = self.maxpool(x)

        return residual_outputs, x

class UpBlock(nn.Module):
    def __init__(self, filters):
        super(UpBlock, self).__init__()
        layers = []
        for i in range(len(filters) - 2):
            layers.append(
                nn.Sequential(
                    ConvBlock(filters[i], filters[i+1]), 
                    nn.ConvTranspose2d(filters[i+1], filters[i+1], 2, stride=2)
                )
            )
        
        layers.append(ConvBlock(filters[-2], filters[-1]))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x, residual_outputs):
        for i in range(len(self.layers)):
            residual = residual_outputs[-(i+1)]
            _, _, h, w = x.shape
            residual = residual[:, :, :h, :w]
            x = torch.cat([x, residual], dim=1)
            x = self.layers[i](x)
        
        return x


In [52]:
db = DownBlock([32, 64, 128], 3)

In [53]:
db

DownBlock(
  (conv_blocks): Sequential(
    (0): ConvBlock(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    )
    (1): ConvBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    )
    (2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    )
  )
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [54]:
a = db(torch.randn(1, 3, 128, 128))
a[1].shape

torch.Size([1, 3, 128, 128])
torch.Size([1, 32, 126, 126])
----------
torch.Size([1, 32, 62, 62])
torch.Size([1, 64, 60, 60])
----------
torch.Size([1, 64, 29, 29])
torch.Size([1, 128, 27, 27])
----------


torch.Size([1, 128, 12, 12])

In [55]:
for residual_output in a[0]:
    print(residual_output.shape)
    print("-"*10)

print(a[1].shape)

torch.Size([1, 32, 124, 124])
----------
torch.Size([1, 64, 58, 58])
----------
torch.Size([1, 128, 25, 25])
----------
torch.Size([1, 128, 12, 12])


In [56]:
bottom_conv = nn.Sequential(ConvBlock(128, 256), nn.ConvTranspose2d(256, 128, 2, stride=2))
bottom_conv(a[1]).shape

torch.Size([1, 128, 12, 12])
torch.Size([1, 256, 10, 10])
----------


torch.Size([1, 128, 16, 16])

In [57]:
up = UpBlock([256, 128, 64, 32])

In [58]:
for layer in up.layers:
    print(layer)
    print("-"*10)

Sequential(
  (0): ConvBlock(
    (conv1): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  )
  (1): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
)
----------
Sequential(
  (0): ConvBlock(
    (conv1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  )
  (1): ConvTranspose2d(64, 64, kernel_size=(2, 2), stride=(2, 2))
)
----------
ConvBlock(
  (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
)
----------


In [59]:
bottom_out = bottom_conv(a[1])
bottom_out.shape


torch.Size([1, 128, 12, 12])
torch.Size([1, 256, 10, 10])
----------


torch.Size([1, 128, 16, 16])

In [60]:
up_out = up(bottom_out, a[0])

torch.Size([1, 256, 16, 16])
torch.Size([1, 128, 14, 14])
----------
torch.Size([1, 192, 24, 24])


RuntimeError: Given groups=1, weight of size [64, 128, 3, 3], expected input[1, 192, 24, 24] to have 128 channels, but got 192 channels instead